In [ ]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

This notebook implements the Thompson sampling algorithm.
First lets look at the pseudo code for the algorithm:

<img src="ab_thompson.png" alt="thompson algorithm" style="width: 800px;"/>

# Beta distributions
Beta distributions have the useful property that the parameters a and b can be interpreted as cointosses:
- a is the number of wins
- b is the number of losses

So let's say we have a "true" coin that has a 40% chance of winning. This "true" chance $p$ is fixed, but our belief over that chance will evolve over time when we gather more observations.

In [ ]:
x = np.linspace(0, 1, 100)
plt.plot(x, stats.beta(4, 6).pdf(x))

Here you can see the beta distribution after 40 wins, 60 losses. Play around with the parameters to see how the distribution changes.

# Multi-armed bandits

These are imaginary gambling machines with multiple arms. The question is: which arm should we pull to maximize our winnings? The problem is that we don't know the probability of winning for each arm. So we have to explore the arms to find out which one is the best. But we also want to exploit the best arm to maximize our winnings. This is the explore-exploit dilemma.

This is a very typical real life scenario. Let's say you have a product, and want to know what the optimal price is for selling the product. Making it cheaper might drive up the price, but you also need to sell more before you get the same profit. Now let's say you are a non-baysian data scientist and you tell the business:

"This is a simple problem. We just need to offer the bike a 1000 times for the low price, and a 1000 times for the high price. Afterwards I can tell you what the best price is." 

Regardless of which of the two prices is the best, the business probably wont be happy with the result: it has cost them a lot of money to find out what the best price is! And a 1000 potential clients have been offered the suboptimal price! Can't we do better than this?

It turns out, yes! We will use bayesian statistics to improve our method and implement the Thompsons sampling algorithm.

# priors
We need beta priors to define our prior belief about the situation. It is very rare that you have really no idea at all. You might ask the business: can you guess, out of 10 clients, how many of them will buy the bike for price a? If you get an answer, this is your prior!
Selling a bike is a win, so 3 out of 10 should be translated into $Beta(3, 7)$. Note that this is NOT the same as $Beta(30, 70)$, which is a much stronger prior. The first prior is very weak, the second is very strong. The first prior will be easily overruled by the data, the second prior will be much harder to overrule with new data.

In [ ]:
@dataclass
class BetaPrior:
    a: int
    b: int
    p: float

Because we dont have a "real world" to get information from, we will need to define the "real" percentage. Normally, you dont have access to this!

In [ ]:
priors = [BetaPrior(3, 7, 0.30), BetaPrior(3, 7, 0.35)]
priors

Now, we are going to pull the "arms" of the "multi armed bandit". Lets create actual beta distributions with our prior beliefs:

In [ ]:
arms = [stats.beta(p.a, p.b) for p in priors]

And lets sample a single value from each of these distributions:

In [ ]:
sample = [bandit.rvs() for bandit in arms]
sample

You should expect a value close to 30% for every arm. Note that this sampling adds "exploration" to the algorithm. We are not just simply (and greedy) exploiting what seems to be the best arm, because we might be fooled by (un)lucky samples! If the distributions are close, we might randomly get the one or the other every time. 

So let's take the best one of these two samples (which involved a bit of luck)

In [ ]:
arm = np.argmax(sample)
arm

So this is the arm that had the luck to be picked for this round. Now lets actually pull the arm with the "real" p value, and see if we actually win or lose:

In [ ]:
real_p = priors[arm].p
stats.bernoulli(real_p).rvs(10)

I pulled it 10 times, so you should expect to see about 3 or 4 wins.
Let's simulation this "real world reward" in a separate class, that can "pay out" rewards after picking the number of an arm

In [ ]:
class Reward:
    def __init__(self, priors: list[BetaPrior]) -> None:
        self.p = [beta.p for beta in priors]
        self.bernoulli = [stats.bernoulli(beta.p) for beta in priors]

    def pay(self, arm: int) -> bool:
        return bool(self.bernoulli[arm].rvs())

    def __repr__(self) -> str:
        return f"Reward({self.p})"


reward = Reward(priors)
reward

In [ ]:
reward.pay(arm)

every time we win, we can update our beliefs by adding a "win" to the a parameter. If we loose, we will add a 1 to the b parameter. This is the "learning" part of the algorithm. We are updating our beliefs based on the data we are gathering.

With all these ingredients, we can wrap them all into a single class that can simulate the multi-armed bandit.

In [ ]:
class MultiArmedBandit:
    def __init__(self, priors: list[BetaPrior]):
        self.arms = [stats.beta(p.a, p.b) for p in priors]
        self.rewards = Reward(priors)

    def __repr__(self) -> str:
        args = [arm.args for arm in self.arms]
        return f"Bandit({args})"

    def pull(self) -> None:
        sample = [arm.rvs() for arm in self.arms]
        # the "luck" part of sampling helps us still explore other arms a bit
        arm = np.argmax(sample)
        reward = self.rewards.pay(arm)

        a, b = self.arms[arm].args
        if reward:
            # winning
            self.arms[arm].args = (a + 1, b)
        else:
            # loosing
            self.arms[arm].args = (a, b + 1)

    def plot(self, ax=None) -> None:
        if ax is None:
            fig, ax = plt.subplots()
        x = np.linspace(0, 1, 100)
        for i in range(len(self.arms)):
            ax.plot(x, self.arms[i].pdf(x), label=f"Bandit {i}")
        ax.legend()

In [ ]:
priors = [BetaPrior(3, 7, p=0.30), BetaPrior(4, 7, p=0.35)]
multiarmedbandit = MultiArmedBandit(priors)
multiarmedbandit.plot()

I changed the priors a bit, so you can have an idea how the two different priors influence the starting point.

In [ ]:
# priors = [BetaPrior(23,27, p=0.45) , BetaPrior(27, 23, p=0.55), BetaPrior(30, 20, p=0.60)]
priors = [BetaPrior(2, 2, p=0.25), BetaPrior(2, 2, p=0.3), BetaPrior(2, 2, p=0.35)]
multiarmedbandit = MultiArmedBandit(priors)

fig, axes = plt.subplots(3, 3, figsize=(12, 8))
axes = axes.ravel()

for i in range(9):
    for _ in range(50):
        multiarmedbandit.pull()
    multiarmedbandit.plot(ax=axes[i])
    axes[i].set_title(f"{str(multiarmedbandit)}")
plt.tight_layout()

total1 = sum(multiarmedbandit.arms[0].args)
total2 = sum(multiarmedbandit.arms[1].args)
total3 = sum(multiarmedbandit.arms[2].args)
total = total1 + total2 + total3

print(
    f"Arm 1 has been pulled {total1} times, {100 * total1 / (total):.2f}% of {total}\n"
    f"Arm 2 has been pulled {total2} times, {100 * total2 / (total):.2f}% of {total}\n"
    f"Arm 3 has been pulled {total3} times, {100 * total3 / (total):.2f}% of {total}\n"
)

Running this experiment, you will see (most of the times! It is still a random method that involves chance...) that bandit 2 with the best percentage will be preferred over the others, typically by a lot!

This is a very elegant way to balance statistical rigor with business needs. We are not just randomly exploring, we are exploring in a smart way, and exploit where that seems justified. This will maximize our winnings considerably over time!